In [ ]:
%tensorflow_version 1.x
!apt install swig cmake libopenmpi-dev zlib1g-dev  > /dev/null 2>&1
!pip install stable-baselines[mpi]==2.10.0 box2d box2d-kengz > /dev/null 2>&1

TensorFlow 1.x selected.


# Lunar Lander environment - SAC vs GAIL

"Landing outside landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt. Four discrete actions available: do nothing, fire left orientation engine, fire main engine, fire right orientation engine. "

Lunar Lander environment: [https://gym.openai.com/envs/LunarLander-v2/](https://gym.openai.com/envs/LunarLander-v2/)

![Lunar Lander](https://cdn-images-1.medium.com/max/960/1*f4VZPKOI0PYNWiwt0la0Rg.gif)




In [ ]:
import gym
import os
import numpy as np

import matplotlib.pyplot as plt

from stable_baselines import SAC, GAIL, results_plotter
from stable_baselines.bench import Monitor
from stable_baselines.gail import ExpertDataset, generate_expert_traj
from stable_baselines.results_plotter import load_results, ts2xy

def evaluate(model, num_steps=1000):
  episode_rewards = [0.0]
  obs = env.reset()
  for i in range(num_steps):
      action, _states = model.predict(obs)
      obs, reward, done, info = env.step(action)
      episode_rewards[-1] += reward
      if done:
          obs = env.reset()
          episode_rewards.append(0.0)
  mean_100ep_reward = round(np.mean(episode_rewards[-100:]), 1)
  print("Mean reward:", mean_100ep_reward, "Num episodes:", len(episode_rewards))
  
  return mean_100ep_reward

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)

# env = gym.make('LunarLander-v2')
env = gym.make('LunarLanderContinuous-v2')

sac_model = SAC('MlpPolicy', env, tensorboard_log="tmp/sac_gail_lunar_log")
mean_reward_before_train = evaluate(sac_model, num_steps=10000)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Mean reward: -129.7 Num episodes: 141


In [ ]:
timesteps = 1e6
sac_model.learn(total_timesteps=int(timesteps), log_interval=10)
# evaluate(sac_model, num_steps=10000)
generate_expert_traj(sac_model, 'sac_lunar_lander_expert', n_timesteps=100, n_episodes=10)
dataset = ExpertDataset(expert_path='sac_lunar_lander_expert.npz', traj_limitation=10)
gail_model = GAIL('MlpPolicy', env, dataset, tensorboard_log="tmp/sac_gail_lunar_log")
gail_model.learn(total_timesteps=int(timesteps), log_interval=10)
# evaluate(gail_model, num_steps=10000)



actions (1954, 2)
obs (1954, 8)
rewards (1954,)
episode_returns (10,)
episode_starts (1954,)
actions (1954, 2)
obs (1954, 8)
rewards (1954,)
episode_returns (10,)
episode_starts (1954,)
Total trajectories: 10
Total transitions: 1954
Average returns: 259.9446582385603
Std for returns: 78.32467212498818








In [ ]:
!zip tmp.zip tmp/* tmp/*/* tmp/*/*/*

  adding: tmp/sac_gail_lunar_log/ (stored 0%)
  adding: tmp/sac_gail_lunar_log/GAIL_1/ (stored 0%)
  adding: tmp/sac_gail_lunar_log/SAC_1/ (stored 0%)
  adding: tmp/sac_gail_lunar_log/SAC_2/ (stored 0%)
  adding: tmp/sac_gail_lunar_log/GAIL_1/events.out.tfevents.1622932717.8338cfb3dc65 (deflated 79%)
  adding: tmp/sac_gail_lunar_log/SAC_1/events.out.tfevents.1622925196.8338cfb3dc65 (deflated 79%)
  adding: tmp/sac_gail_lunar_log/SAC_2/events.out.tfevents.1622932713.8338cfb3dc65 (deflated 90%)
